In [1]:
import QuantLib as ql

In [2]:
calendar = ql.TARGET()

In [3]:
todaysDate = ql.Date(15, ql.May, 2007)

In [4]:
todaysDate = calendar.adjust(todaysDate)

In [5]:
todaysDate

Date(15,5,2007)

In [6]:
ql.Settings.instance().evaluationDate = todaysDate

In [7]:
flatRate = ql.SimpleQuote(0.01)

In [8]:
tsCurve = ql.YieldTermStructureHandle(ql.FlatForward(todaysDate, ql.QuoteHandle(flatRate), ql.Actual365Fixed()))

In [9]:
recovery_rate = 0.5

In [10]:
quoted_spreads = [0.0150, 0.0150, 0.0150, 0.0150 ]

In [11]:
tenors = ql.PeriodVector()
tenors.append(ql.Period("3M"))
tenors.append(ql.Period("6M"))
tenors.append(ql.Period("1Y"))
tenors.append(ql.Period("2Y"))


In [12]:
maturities = [calendar.adjust(todaysDate + tenors[i], ql.Following) for i in range(0,4)]

In [13]:
instruments = ql.DefaultProbabilityHelperVector()
for i in range(0, 4):
    instruments.append(ql.SpreadCdsHelper(ql.QuoteHandle(ql.SimpleQuote(quoted_spreads[i])),
                                        tenors[i], 0, calendar, ql.Quarterly, ql.Following,
                                        ql.DateGeneration.TwentiethIMM, ql.Actual365Fixed(),
                                        recovery_rate, tsCurve))

In [14]:
nominal = 1000000.0

In [15]:
cdsSchedule = ql.MakeSchedule(todaysDate, calendar.adjust(todaysDate + tenors[0], ql.Following),ql.Period(ql.Quarterly),
                             ql.Quarterly, calendar, ql.Following, ql.Unadjusted, ql.DateGeneration.TwentiethIMM)

In [16]:
for d in cdsSchedule:
    print(d)

May 15th, 2007
June 20th, 2007
September 20th, 2007


In [17]:
cds_3m = ql.CreditDefaultSwap(ql.Protection.Seller, nominal, quoted_spreads[0],
                             cdsSchedule, ql.Following, ql.Actual365Fixed())

In [18]:
hazardRateStructure = ql.PiecewiseFlatHazardRate(
        todaysDate, instruments, ql.Actual365Fixed())

In [19]:
probability = ql.DefaultProbabilityTermStructureHandle(hazardRateStructure)

In [20]:
engine = ql.MidPointCdsEngine(probability, recovery_rate, tsCurve)

In [21]:
cds_3m.setPricingEngine(engine)

In [22]:
print("3M fair spread: ",cds_3m.fairSpread())
print("   NPV:         ", cds_3m.NPV())
print("   default leg: ", cds_3m.defaultLegNPV())
print("   coupon leg:  ", cds_3m.couponLegNPV())

3M fair spread:  0.015000000000000206
   NPV:          -7.185008144006133e-11
   default leg:  -5218.163413350623
   coupon leg:   5218.163413350551


In [23]:
cds_3m.upfrontNPV()

0.0

In [24]:
cds_3m.upfront()

In [25]:
cds_3m.fairUpfront()

7.18540185386899e-17

In [26]:
cds_3m2 = ql.CreditDefaultSwap(ql.Protection.Seller, nominal, 1000., 0.05, 
                             cdsSchedule, ql.Following, ql.Actual365Fixed(),
                              False, True, todaysDate, todaysDate)

In [27]:
cds_3m2.setPricingEngine(engine)

In [28]:
print("3M fair spread: ",cds_3m2.fairSpread())
print("   NPV:         ", cds_3m2.NPV())
print("   default leg: ", cds_3m2.defaultLegNPV())
print("   coupon leg:  ", cds_3m2.couponLegNPV())

3M fair spread:  0.015047101901310615
   NPV:          12121.266622994506
   default leg:  -5218.163413350623
   coupon leg:   17339.43003634513


In [29]:
cds_3m2.upfront()

1000.0